In [28]:
import numpy as np
np.set_printoptions(suppress=True) 

In [29]:
class Logistic():
    '''Implementacja modelu regresji logistycznej trenowana poprzez użycie SGD z momentum '''

    def __init__(self):
        self.theta = None
        self.conf_matrix = None

    
    def sigmoid(self,X:np.ndarray) -> float:
        return 1/(1+np.exp(-X))
    
 
    def fit(self, X: np.ndarray, y: np.ndarray,iterations=10,learning_rate=0.01,gamma=0.9,C=1.0) ->None:
        '''Zdecydowałem się na użycie regularyzacji, aby poprawić wyniki algorytmu. Jej mocą steruje się poprzez odpowiednie ustawienie parametru C,
            domyślnie jest on ustawiony na 1. Ponadto można ustawić ilość iteracji algorytmu, tempo ucznenia, a także moc momentum. '''
        assert C !=0
        m = X.shape[0]
        n = X.shape[1]+1
        X_b = np.c_[np.ones((m,1)),X]
        velocity = 0 
        self.theta = np.random.randn(n,1) 
        lamb = 1/C
        for iteration in range(iterations):
            for i in range(m):
                random_index = np.random.randint(m)
                xi = X_b[random_index:random_index+1]
                yi = y[random_index:random_index+1]
                sigm = self.sigmoid(xi @ self.theta)
                gradient = xi.T.dot (sigm - yi) + (lamb/m)*(self.theta)
                velocity = gamma * velocity
                speed = learning_rate*gradient +velocity
                self.theta = self.theta - speed
        return
    
    def coef(self) -> np.ndarray:
        '''Funkcja zwracająca parametry wytrenowane przez model '''
        return self.theta
    
    def validate(self,X: np.ndarray,y:np.ndarray,cv=5,C=1) -> str:
        '''Funkcja wykonująca kroswalidacje poprzez podzielenie zbioru na cv podzbiorów, a nastepnie iteracyjne tranowanie modelu na cv-1 częsciach, a 
            następnie sprawdzanie wyników na pozostałym zbiorze. W efekcie mamy cv wyników które są uśredniane i zwracane przez funkcje '''
        size = X.shape[0]
        scores = []
        perm = np.random.permutation(size)
        X_b = X[perm]
        y_b = y[perm]
        X_b = np.array(np.array_split(X_b,cv))
        y_b = np.array(np.array_split(y_b,cv))
        mask = np.ones(cv,dtype=bool) #Pomocnicza tablica do wybierania odpowiednich podtablic
        for i in range(cv):
            mask[i] = 0
            X_val = X_b[mask==0][0]
            y_val = y_b[mask==0][0]
            res = X_b.shape[0] -y_val.shape[0]
            X_fit = X_b[mask].reshape(res,(28*28))
            y_fit = y_b[mask].reshape(res,)
            self.fit(X_fit,y_fit,C=C)
            pred = self.predict(X_val)
            score =  self.evaluate(y_val,pred)
            scores.append(score)
            mask[i] = 1
        return f'Mean = {np.mean(scores)}, Std = {np.std(scores)}'
            

    def predict(self, X: np.ndarray) -> np.ndarray:
        '''Funckja do przewidywania na podstawie wcześniej wytrenowanych parametrów'''
        m = X.shape[0]
        X_b = np.c_[np.ones((m,1)),X]
        return np.round(self.sigmoid(X_b @ self.theta))
    

    def evaluate(self,y_true: np.ndarray, y_pred: np.ndarray) ->float:
        '''Funckja mierząca accuracy '''
        assert y_true.shape[0] == y_pred.shape[0]
        TP = 0
        FP = 0
        TN = 0
        FN = 0
        for i in range(len(y_true)):
            if y_true[i]==1 and y_true[i] == y_pred[i]:
                TP +=1
            elif y_true[i]==0 and y_true[i] == y_pred[i]:
                TN +=1
            elif  y_true[i] > y_pred[i]:
                FN+=1
            elif y_pred[i]>y_true[i]:
                FP +=1
        accuracy = (TP+TN)/(TP+FP+TN+FN)
        self.conf_matrix = np.array([[TP,FP],[FN,TN]])
        return accuracy

    def confusion(self) ->np.ndarray:
        '''Funkcja zwracająca macierz pomyłek '''
        return self.conf_matrix
        




In [30]:
def loadMNIST( prefix, folder ):
    '''Funkcja do ładowania zbiou danych z pliku ubyte '''
    intType = np.dtype( 'int32' ).newbyteorder( '>' )
    nMetaDataBytes = 4 * intType.itemsize

    data = np.fromfile( folder + "/" + prefix + '-images-idx3-ubyte', dtype = 'ubyte' )
    magicBytes, nImages, width, height = np.frombuffer( data[:nMetaDataBytes].tobytes(), intType )
    data = data[nMetaDataBytes:].astype( dtype = 'float32' ).reshape( [ nImages, width, height ] )

    labels = np.fromfile( folder + "/" + prefix + '-labels-idx1-ubyte',
                          dtype = 'ubyte' )[2 * intType.itemsize:]

    return data, labels


def delete_elems(X,y):
    '''Usunięcie elementów zawierających 0 bądź 1, zgodnie z poleceniem'''
    index = np.argwhere(np.logical_or(y==0,y==1))
    X = np.delete(X,index,axis=0)
    y= np.delete(y,index)
    return X,y


def change_labels(y):
    '''Odpowiednie ustawienie labeli, tak że 1 odpowiada liczbom pierwszym, a 0 liczbom złożonym '''
    index_prime = np.argwhere(np.logical_or.reduce((y==2,y==3,y==5,y==7)))
    index_composite = np.argwhere(np.logical_or.reduce((y==4,y==6,y==8,y==9)))
    y[index_prime] = 1
    y[index_composite] = 0 
    return y

def resize(X):
    '''Przeskalowanie zbioru danych, do przedziału [0,1] '''
    n = X.shape[0]
    X = X.reshape(n,28*28)
    X = X/255
    return X

def shuffle(X,y):
    '''Funkcja do losowego przemieszania zbioru '''
    perm= np.random.permutation(X.shape[0])
    X = X[perm]
    y = y[perm]
    return X,y



In [31]:
def prepare():
    '''Funckja służąca jako wraper, przygotowujący zbiór danych do odpalenia na nim modelu '''
    X_train, y_train = loadMNIST( "train", "./Allegro_data")
    X_test, y_test = loadMNIST( "t10k", "./Allegro_data" )
    X_train,y_train = delete_elems(X_train,y_train)
    X_test,y_test = delete_elems(X_test,y_test)
    y_train = change_labels(y_train)
    y_test = change_labels(y_test)
    X_train = resize(X_train)
    X_test = resize(X_test)
    X_train,y_train = shuffle(X_train,y_train)
    X_test,y_test = shuffle(X_test,y_test)
    return X_train,X_test,y_train,y_test


W celu uzyskania jak najlepszych parametrów zdecydowałem się na użycie walidacji krzyżowej. Zdecydowałem się robić walidację jedynie zmieniając parametr C, uznając pozostałe jako stałe elementy SGD, tzn.jako takie których nie warto modyfikować.

In [40]:
C_val = [0.01,0.1,1,10,100]
scores = []
for i in range(len(C_val)):
    X_train,X_test,y_train,y_test = prepare()
    logistic = Logistic()
    score = logistic.validate(X_train,y_train,C = C_val[i])
    tup = (score,f'C = {C_val[i]}')
    scores.append(tup)
print(*scores, sep='\n')

('Mean = 0.9028625752614344, Std = 0.003584332408898654', 'C = 0.01')
('Mean = 0.9042146403295657, Std = 0.0046975825152130194', 'C = 0.1')
('Mean = 0.9056934614978346, Std = 0.00584064637629604', 'C = 1')
('Mean = 0.9070666525826555, Std = 0.004567521025467034', 'C = 10')
('Mean = 0.9062638639484526, Std = 0.0039026563570105244', 'C = 100')


Widać że wyniki są stabilne, praktycznie nie występują odchylenia w wyniku w trakcie zmian parametru regularyzacji.

In [42]:
X_train,X_test,y_train,y_test = prepare()
logistic = Logistic()
logistic.fit(X_train,y_train)
pred = logistic.predict(X_test)
score =logistic.evaluate(y_test,pred)
logistic.confusion()

array([[3624,  315],
       [ 338, 3608]])

Macierz pomyłek potwierdza poprzednie dobre wyniki, jednocześnie pokazując, że istnieje pole do poprawy,jednakże ciężko będzie to osiągnąc tym algorytmem.

In [43]:
score

0.9171845275840202

Uważam iż udało mi się osiągnąc dobre wyniki. Accuracy na zbiorze testowym na poziomie 91-92% jest wynikiem porównywalnym z wynikiem, który osiągnęła implementacja tego samego algorytmu w bibliotece Scikit-Learn. Mogę stwierdzić iż regularyzacja, oraz użycie momentum znacząco poprawiło działanie algorytmu. Obie metody poprawiły accuracy, a także pozwoliły ustabilizować wyniki. Jeśli chodzi o analizę błędów popełnianych przez algorytm, to jest to dość utrudnione ze wzgłędu na sformułwanie problemu dzielące zbiór na liczby pierwsze i złożone, przez co, nie można zdecydowanie stwierdzić na jakich cyfrach algorytm myli się najczęsciej. Aby osiągnąć lepsze accuracy należałoby użyć algorytmu zdolnego analizować bardziej skomplikowane wzorce, jak na przykład SVM.